In [2]:
import pandas as pd
import numpy as np

In [3]:
!ls /datasets/bikedata

202109-divvy-tripdata.csv  202201-divvy-tripdata.csv  202205-divvy-tripdata.csv
202110-divvy-tripdata.csv  202202-divvy-tripdata.csv  202206-divvy-tripdata.csv
202111-divvy-tripdata.csv  202203-divvy-tripdata.csv  202207-divvy-tripdata.csv
202112-divvy-tripdata.csv  202204-divvy-tripdata.csv  202208-divvy-tripdata.csv


In [6]:
sept_2021 = pd.read_csv("/datasets/bikedata/202109-divvy-tripdata.csv")

In [7]:
sept_2021.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,9DC7B962304CBFD8,electric_bike,2021-09-28 16:07:10,2021-09-28 16:09:54,NaN,NaN,NaN,NaN,41.89,-87.68,41.89,-87.67,casual
1,F930E2C6872D6B32,electric_bike,2021-09-28 14:24:51,2021-09-28 14:40:05,NaN,NaN,NaN,NaN,41.94,-87.64,41.98,-87.67,casual
2,6EF72137900BB910,electric_bike,2021-09-28 00:20:16,2021-09-28 00:23:57,NaN,NaN,NaN,NaN,41.81,-87.72,41.80,-87.72,casual
3,78D1DE133B3DBF55,electric_bike,2021-09-28 14:51:17,2021-09-28 15:00:06,NaN,NaN,NaN,NaN,41.80,-87.72,41.81,-87.72,casual
4,E03D4ACDCAEF6E00,electric_bike,2021-09-28 09:53:12,2021-09-28 10:03:44,NaN,NaN,NaN,NaN,41.88,-87.74,41.88,-87.71,casual


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82a9fd68-25b4-43bb-8b8d-79b5852e5f33' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>